<a href="https://colab.research.google.com/github/Zack1243/Aniyomi-Downloader-through-links-file/blob/main/Aniyomi_local_anime_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [46]:
import requests
import json
import os
from PIL import Image
import re
import subprocess
import sys
!python3 -m pip install -U --pre "yt-dlp[default]"
import yt_dlp

## Filter json file

In [59]:
def filter_details(file):
  default_options = {
    "title": "",
    "author": "",
    "artist": "",
    "description": "",
    "genre": [],
    "status": "0"
  }
  with open(file, 'r') as f:
    data = json.load(f)
    transformed_data = {
        "title": data.get("title", ""),
        "author": data.get("uploader", ""),
        "artist": data.get("uploader", ""),
        "description": data.get("description", ""),
        "genre": data.get("tags", []),
        "status": data.get("status", "0")
    }

    url_pattern = r'https?://\S+|www\.\S+'

    # Replace URLs with an empty string
    transformed_data["description"] = re.sub(url_pattern, '', transformed_data["description"])

    with open(file, 'w') as f:
      json.dump(transformed_data, f)

## Extract url title and make directory for it

In [99]:
def downloader(url):
    try:
        # Get the video title
        result = subprocess.run([
            'yt-dlp',
            '--skip-download',
            '--get-title',
            url
        ], capture_output=True, text=True, check=True)

        # Extract the title from the result
        title = result.stdout.strip()

        # Make path to new directory including video's title
        path = os.path.join(os.curdir, title)

        # Check if path exists, else make one
        if not os.path.exists(path):
            os.makedirs(path)

        # Make a list for downloading the video
        subprocess.run([
            'yt-dlp',
            '--convert-thumbnails', 'png',
            '--write-thumbnail',
            '--embed-thumbnail',
            '--add-metadata',
            '--write-info-json',
            '-f', 'bestvideo[height<=1080]+bestaudio/best[height<=1080]/best',
            '-o', f'{path}/%(title)s.%(ext)s',
            url
        ], check=True)

        # Change .png file to be "cover.png"
        if os.path.exists('%(title)s.png'):
          os.rename('%(title)s.png', 'cover.png')

        # Change info.json file to be "details.json"
        if os.path.exists('%(title)s.info.json'):
          os.rename('%(title)s.info.json', 'details.json')
          # Rewrite details.json to aniyomi style guidelines
          filter_details('details.json')

        # Return to correct directory
        os.chdir('..')

    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

## Example of aniyomi style .json file


```
{
	"title": "Example Title",
	"author": "Example Author",
	"artist": "Example Artist",
	"description": "Example Description",
	"genre": ["genre 1", "genre 2", "etc"],
	"status": "0",
	"_status values": ["0 = Unknown", "1 = Ongoing", "2 = Completed", "3 = Licensed", "4 = Publishing finished", "5 = Cancelled", "6 = On hiatus"]
}
```


## Get urls

In [97]:
# Get a list of urls
urls = input("Enter urls separated by space or enter: ")
#urls = 'https://www.youtube.com/watch?v=oDZK3_c-Vkc'
downloader(urls)
# Parse through urls
urls = urls.split()

Enter urls separated by space or enter: https://www.youtube.com/watch?v=oDZK3_c-Vkc


In [ ]:
#https://github.com/ytdl-org/youtube-dl/tree/master?tab=readme-ov-file#use-convenience-conversion-and-parsing-functions


# Get the metadata from a url
#meta = self._download_json(url, video_id)

# Extract the summary
#description = meta.get('summary')

# Useful if summary not present
#description = self._search_regex(
    #r'<span[^>]+id="title"[^>]*>([^<]+)<',
    #webpage, 'description', default=None)

# This gets the title, or if fails can get the webpage's title
#title = meta.get('title') or self._og_search_title(webpage)
